# T4 - Final Project

Semester 2221, CSEC 520/620, Team 4\
Final Project - URL Classification\
Due by December 14, 2022 11:59 PM EST.\
Accounts for 18% of total grade.

In [1]:
# Ensure the WHOIS package is installed
print(f'{"":#^{36}}\n{"## Installing Packages ":#<{36}}\n{"":#^{36}}')
!pip install python-whois

# Download our repo, which contains the RAW datasets
print(f'\n{"":#^{36}}\n{"## Updating Repository ":#<{36}}\n{"":#^{36}}')
!git init
!git remote add origin https://github.com/aisgbnok/T4-Project.git
!git pull origin main --allow-unrelated-histories

####################################
## Installing Packages #############
####################################

####################################
## Updating Repository #############
####################################
Reinitialized existing Git repository in C:/Users/Anthony/Documents/T4-Project/.git/


error: remote origin already exists.


Already up to date.


From https://github.com/aisgbnok/T4-Project
 * branch            main       -> FETCH_HEAD


In [2]:
import os
import pandas as pd
import whois

In [3]:
def load_data(seed=75, output=False, save=False):
  """
  Loads our separate dataframes and merges them together.
  Ensures the columns are equal, normalizes the labels,
  and finally shuffles the columns using the seed.

  :param seed: Integer value that ensures reproduction of resulting dataframe.
  :param output: Whether to print dataframes or not. True or False.
  :param save: Whether to save the pandas dataframe. Can be False (default), 'CSV', or 'PICKLE'.
  :return: A pandas dataframe that contains a url and label column.
           The label column is 0 for benign and 1 for malicious.
  """
  # Get all of our datasets
  df_aj = pd.read_csv(os.path.join('datasets', 'urls-antonyj.csv'))
  df_ms = pd.read_csv(os.path.join('datasets', 'urls-manu-siddhartha.csv'))

  if output:
    print(f'{"":#^{36}}\n{"## Original ":#<{36}}\n{"":#^{36}}')
    print('## urls-antonyj.csv')
    display(df_aj)
    print('## urls-manu-siddhartha.csv')
    display(df_ms)

  # Ensure Columns Match
  df_ms.columns = df_aj.columns

  # Merge dataframes
  df = pd.merge(df_aj, df_ms, how='outer', on='url')

  # Normalize Data, 1 is malicious, 0 is benign
  df['label_x'] = (df['label_x'] == 'bad').astype(int)
  df['label_y'] = (df['label_y'] != 'benign').astype(int)

  # Combine labels
  df.label_x.fillna(df.label_y, inplace=True)
  df = df.rename(columns={'label_x': 'label'})
  df = df.drop('label_y', axis=1)

  # Keep first exact matches
  df = df.drop_duplicates()

  # Drop all duplicate urls with different types; bad data
  df = df.drop_duplicates(subset='url', keep=False)

  # Shuffle using seed value
  df = df.sample(frac=1, random_state=seed)

  # Reset Index
  df = df.reset_index(drop=True)

  if output:
    print(f'{"":#^{36}}\n{"## Resulting ":#<{36}}\n{"":#^{36}}')
    print('## urls-antonyj.csv')
    display(df_aj)
    print('## urls-manu-siddhartha.csv')
    display(df_ms)
    print('## Final')
    display(df)

  if save == 'PICKLE':
    df.to_pickle('datasets/t4-dataset.zip')
  elif save == 'CSV':
    df.to_csv('datasets/t4-dataset.csv')

  return df

In [4]:
dataset = load_data()

In [ ]:
import threading


#multithreading the data set
def multithreading():
  perc = 0.0
  threads = []
  max_threads = 10
  thread_num = 1
  while perc < 1:
    if len(threads) >= max_threads:
      for thread in threads:
        thread.join()
        print("Thread "+str(thread_num)+" done")
        thread_num+=1
    print("perc "+str(perc))
    thread = threading.Thread(target=checker, args=(perc*100, dataset.iloc[int(perc*len(dataset)):int((perc+0.01)*len(dataset))]))
    thread.start()
    threads.append(thread)
    perc = perc + 0.01

In [ ]:
#numsection -> the percent into the data that it is
#data -> the data of the current section
def checker(numsection, data):
  rows = []
  for index, row in data.iterrows():
    w = None
    try:
      w = whois.whois(data.iloc[index]["url"])
    except:
      continue
    if w.domain_name != "null" and w.domain_name is not None:
      rowdict = {}
      rowdict["originalurl"] = data.iloc[index]["url"]
      rowdict.update(dict(w))
      rows.append(rowdict)
  whoisframe = pd.DataFrame(rows)
  whoisframe.to_csv('csv/whoisdata' + str(numsection) + '.csv')
  print("csv made")

In [ ]:
# Import the drive module from the Google Colab library
from google.colab import drive

# Mount your personal Google Drive
drive.mount('/content/drive/')

# Immediately change the current directory to the shared drive.
# This will reduce the chance that your personal drive will be modified erroneously.
os.chdir('/content/drive/Shareddrives/CSEC 620 Group 4/Final Project')

In [ ]:
#multithreading()

In [ ]:
os.chdir('/content/drive/Shareddrives/CSEC 620 Group 4/Final Project/csv')
filelist = os.listdir('.')
newframe = pd.DataFrame()
for f in filelist:
  if f.find('whoisdata') == 0:
    currentframe = pd.read_csv(f)
    newframe = pd.concat([newframe, currentframe], ignore_index=True)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (48,55,82,84,86,87,88,89,90,91,92,93,94,95,96,97,105,106,107,108,111,112,118,119,124,125,126,131,139,146,147,148,149,150,151,152) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
nummal = 0
numbenign = 0
for index, row in newframe.iterrows():
  thisurl = row["originalurl"]
  mergedrow = dataset.loc[dataset["url"] == thisurl]
  benigncheck = mergedrow.iloc[0]["label"]
  if benigncheck == "good":
    numbenign = numbenign + 1
  else:
    nummal = nummal + 1
print(nummal)
print(numbenign)

2712
3588


# Splitting the data set

In [ ]:
splitVal = int(len(newframe)*.2)
testing_set = newframe.iloc[:splitVal,]
training_set = newframe.iloc[splitVal:,]

In [ ]:
countries = {}
for index, row in training_set.iterrows():
  rowcountry = row["country"]
  if rowcountry in countries.keys():
    countries[rowcountry] = countries[rowcountry] + 1
  else:
    countries[rowcountry] = 1

for c in countries:
  print(c, countries[c])

US 2285
nan 1953
ID 2
CN 125
AU 8
GB 69
IS 92
BG 12
TR 4
CM 1
CA 127
AT 2
SG 5
['CA', 'US'] 1
CY 9
SK 3
BR 39
LU 4
SE 4
JP 19
UK 10
KN 4
PL 6
IT 13
DE 26
REDACTED FOR PRIVACY 32
IN 18
Malaysia 1
RU 9
FR 23
United States 2
SI 1
NZ 1
Canada 2
Austria 6
CR 3
KR 5
ES 10
HU 2
RO 12
HK 6
KY 3
PA 8
cn 2
SPAIN 1
UA 1
['CA', 'IT'] 2
TH 3
NL 9
CZ 1
AL 1
IL 1
SN 3
EC 2
MK 1
SC 2
BE 4
['US', 'REDACTED FOR PRIVACY'] 1
China 10
VN 4
ar 1
PS 1
CO 1
MX 1
HK,CN 1
PT 3
CH 6
EE 1
PK 1
IR 6
PH 1
AE 1
['Austria', 'Germany'] 1


In [ ]:
regs = {}
for index, row in training_set.iterrows():
  rowreg = row["registrar"]
  if rowreg in regs.keys():
    regs[rowreg] = regs[rowreg] + 1
  else:
    regs[rowreg] = 1

for r in regs:
  print(r, regs[r])

MarkMonitor Inc. 185
GoDaddy.com, LLC 1506
IONOS SE 22
TurnCommerce, Inc. DBA NameBright.com 27
MarkMonitor, Inc. 425
Network Solutions, LLC 300
Exonet B.V. 2
TUCOWS, INC. 75
Amazon Registrar, Inc. 82
nan 223
Tucows Domains Inc. 44
CSC CORPORATE DOMAINS, INC. 184
eName Technology Co.,Ltd. 4
NAMECHEAP INC 126
Ever Ready Names, Inc 3
US Locality 2
Moniker Online Services LLC 7
easyDNS Technologies Inc. 12
Webnames.ca Inc. 5
FastDomain Inc. 15
Hetzner Online GmbH 5
Key-Systems GmbH 28
SYNERGY WHOLESALE ACCREDITATIONS PTY LTD 2
Alibaba Cloud Computing (Beijing) Co., Ltd. 215
Wild West Domains, LLC 18
DomainSpot LLC 3
RegistrarSafe, LLC 82
DNC Holdings, Inc 19
ENOM, INC. 96
Sea Wasp, LLC 19
Google LLC 36
Registrar.eu 5
Hostinger, UAB 10
ТОВ "Центр Інтернет-Імен України" 1
1API GmbH 12
KIFCORP 1
GRANSY S.R.O D/B/A SUBREG.CZ 1
Gname.com Pte. Ltd. 71
MarkMonitor International Canada Ltd. 3
EuroDNS S.A. 2
NameCheap, Inc. 16
Name.com, Inc. 25
GoDaddy Corporate Domains, LLC 30
2F Communication s.

In [ ]:
dns = {}
for index, row in training_set.iterrows():
  rowdns = row["dnssec"]
  if rowdns in dns.keys():
    dns[rowdns] = dns[rowdns] + 1
  elif 'unsigned' in str(rowdns).lower() and (rowdns != 'unsigned'):
    dns["unsigned"] = dns["unsigned"] + 1
  else:
    dns[rowdns] = 1

for r in dns:
  print(r, dns[r])

unsigned 3867
nan 1071
yes 17
signedDelegation 48
Inactive 12
Signed delegation 3
no 19
signed delegation 3


In [ ]:
from numpy.lib.function_base import percentile
percentages = {}
percentagerow = []

for index, row in training_set.iterrows():
  rowdomain = row["domain_name"]
  num = 0
  lett = 0
  for char in rowdomain:
    if char.isalpha():
      lett += 1
    elif char.isnumeric():
      num += 1
  perc = round(num/(num+lett), 1)
  percentagerow.append(perc)
  if perc in percentages.keys():
    percentages[perc] = percentages[perc] + 1
  else:
    percentages[perc] =  1

for p in percentages:
  print(p, percentages[p])

0.0 3962
0.1 50
0.5 624
0.2 38
0.3 20
0.4 252
0.7 87
0.8 7


In [ ]:
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
le1 = preprocessing.LabelEncoder()
url1 = le1.fit_transform(training_set["originalurl"])
countrylabel = le1.fit_transform(training_set["country"])
model1 = GaussianNB()
model1.fit(url1.reshape(-1, 1), countrylabel)

GaussianNB()

In [ ]:
le2 = preprocessing.LabelEncoder()
url2 = le2.fit_transform(training_set["originalurl"])
reglabel = le2.fit_transform(training_set["registrar"])
model2 = GaussianNB()
model2.fit(url2.reshape(-1, 1), reglabel)

GaussianNB()

In [ ]:
le3 = preprocessing.LabelEncoder()
url3 = le3.fit_transform(training_set["originalurl"])
dnslabel = le3.fit_transform(training_set["dnssec"])
model3 = GaussianNB()
model3.fit(url3.reshape(-1, 1), dnslabel)

GaussianNB()

In [ ]:
le4 = preprocessing.LabelEncoder()
url4 = le4.fit_transform(training_set["originalurl"])
percentlabel = le4.fit_transform(percentagerow)
model4 = GaussianNB()
model4.fit(url4.reshape(-1, 1), percentlabel)

GaussianNB()